In [1]:
import pandas as pd
import urllib

In [2]:
election_office_url = "https://www.sos.state.mn.us/elections-voting/find-county-election-office/#"

response = urllib.request.urlopen(election_office_url)
webContent = response.read()

f = open('mn_election_offices-20200816.html', 'wb')
f.write(webContent)
f.close

<function BufferedWriter.close>

In [3]:
def get_clean_line(line):
    '''Return list of clean words from the string line'''
    import re, string
    clean_line = re.sub(r"[><]", " ", line)   # help with cleaning
    return clean_line.split()


counties = []
election_officials = []
general_phone_numbers = []
addresses = []
absentee_voting_contacts = []
absentee_voting_contact_phone_numbers = []
military_overseas_contacts = []
military_overseas_contact_phone_numbers = []


file_str = 'mn_election_offices-20200816.html'

with open(file_str, 'r') as local_page:
    
    line_num = 0

    for line in local_page:
        


        if line_num >= 400:
#             print(line_num, ":", line, end="")

            # Get the counties
            if 'h2 class' in line:
#                 print(line_num, ":", line, end="")
                line_lst = get_clean_line(line)
                matching = [item for item in line_lst if "data-target" in item]
                counties.append(" ".join(line_lst[line_lst.index(matching[0])+1:line_lst.index("County")+1]))
            
            
            # Get the Election Official's name and phone number
            if 'official:' in line:
                line_lst = get_clean_line(line)
#                 print(line_num, ":", line_lst)
                election_officials.append(" ".join(line_lst[line_lst.index('official:')+1:line_lst.index("br")]))
                general_phone_numbers.append(line_lst[line_lst.index('phone:')+1])
            
            
            # Get Address
            if 'Address</h3>' in line:
#                 print(line_num, ":", line, end="")
                line = local_page.readline()
                line_lst = line.split('<br />')
                line_num += 1
#                 print(line_num, ":", line_lst)
                
                # Deal with issue with Winona address
                if 'Winona' in line:
                    line = local_page.readline()
                    line_num += 1
#                     print("New line number: ", line_num, ":", line)
                    line_lst = line.replace('<div>', "").split('<br />')
                    addresses.append(" ".join(line_lst).strip())
#                     print(line_num, ":", addresses[-1])

                # Deal with issue with Scott county
                elif "Mailing" in line:
                    for ii in range(4):
                        line = local_page.readline()
                        line_num += 1
                    line1 = local_page.readline().replace('</div>', "").replace('<div>', "").strip()
                    line_num += 1
#                     print("Newer line number: ", line_num, ":", line)
                    line2 = local_page.readline().replace('</div>', "").replace('<div>', "").strip()
                    line_num += 1
                    line_lst = [line1, line2]
                    addresses.append(" ".join(line_lst))
#                     addresses.append("Hello There")


                # Deal with issue with Stearns address
                elif 'Stearns' in line:
                    line = local_page.readline()
                    line_num += 1
#                     print("Newer line number: ", line_num, ":", line)
                    line_lst = line.split('<br />')
                    addresses.append(" ".join(line_lst[1:]).strip())

                else:
                    addresses.append(" ".join(line_lst[1:]).strip())
#                     print(line_num, ":", addresses[-1])
            if 'Street address</h3>' in line:
#                 print(line_num, ":", line, end="")
                line = local_page.readline()
                line_lst = line.split('<br />')
                addresses.append(" ".join(line_lst[1:]).strip())
                line_num += 1
#                 print(line_num, ":", addresses[-1])
            
            
            # Get Absentee voting contact's name and phone number
            if 'Absentee voting contact' in line:
#                 print(line_num, ":", line, end="")
                line_lst = get_clean_line(local_page.readline())
                line_num += 1
#                 print(line_num, ":", line_lst)
                absentee_voting_contacts.append(" ".join(line_lst[:line_lst.index('br')]).replace("/span","").replace("span","").strip())
                absentee_voting_contact_phone_numbers.append(line_lst[line_lst.index("Phone:")+1])
        
        
            # Get military and oversea's voting contact's name and phone number
            if 'Military' in line:
#                 print(line_num, ":", line, end="")
                line_lst = get_clean_line(local_page.readline())
                line_num += 1
#                 print(line_num, ":", line_lst)
                military_overseas_contacts.append(" ".join(line_lst[:line_lst.index('br')]).replace("/span","").replace("span","").strip())
                military_overseas_contact_phone_numbers.append(line_lst[line_lst.index("Phone:")+1])

             
            
        line_num += 1
        
        
df = pd.DataFrame(
    list(zip(
        counties,
        election_officials,
        general_phone_numbers,
        addresses, 
        absentee_voting_contacts, 
        absentee_voting_contact_phone_numbers,
        military_overseas_contacts, 
        military_overseas_contact_phone_numbers)),
    columns =['county', 'election_official', 'general_phone_number', 'address', 'absentee_voting_contact',
              'absentee_voting_contact_phone_number', 'military_and_overseas_contact', 'military_and_overseas_contact_phone_number']
                 )

df

,county,election_official,general_phone_number,address,absentee_voting_contact,absentee_voting_contact_phone_number,military_and_overseas_contact,military_and_overseas_contact_phone_number
0,Aitkin County,Kirk Peysar,218-927-7354,"307 2nd St NW Rm 121 Aitkin, MN 56431",Sally Huhta,218-927-7354,Sally Huhta,218-927-7354
1,Anoka County,Paul Linnell,763-324-1300,"2100 3rd Ave, Suite W130 Anoka, MN 55303-5031",Anoka County Elections,763-324-1300,Anoka County Elections,763-324-1300
2,Becker County,Mary E. Hendrickson,218-846-7311,"915 Lake Ave Detroit Lakes, MN 56501",Becker County Elections,218-846-7311,Mary E. Hendrickson,218-846-7311
3,Beltrami County,JoDee Treat,218-333-4175,"701 Minnesota Ave NW Ste 220 Bemidji, MN 5660...",Danielle Johnson,218-333-8476,Danielle Johnson,218-333-8476
4,Benton County,Nadean Inman,320-968-5027,"531 Dewey St Foley, MN 56329-0129",Heather Bondhus,320-968-5027,UOCAVA,320-968-5027
...,...,...,...,...,...,...,...,...
82,Watonwan County,Kelly Pauling,507-375-1210,"710 2nd Ave S Saint James, MN 56081",Kelly Pauling,507-375-1210,Kelly Pauling,507-375-1210
83,Wilkin County,Janelle Krump,218-643-7165,"300 S 5th St Breckenridge, MN 56520",Brenda Conzemius,218-643-7165,Brenda Conzemius,218-643-7165
84,Winona County,Sandra J Suchla,507-457-8830,"202 W Third St Winona, MN 55987",Sandra J Suchla,507-457-8830,Sandra J Suchla,507-457-8830
85,Wright County,Robert (Bob) Hiivala,763-682-7579,"10 2nd St NW Rm 230 Buffalo, MN 55313-1195",Lanette Paumen,763-682-7692,Lanette Paumen,763-682-7692


In [4]:
standardized_df = pd.DataFrame(
    list(zip(
        ['Minnesota'] * len(counties),
        ['MN'] * len(counties),
        counties,
        ['County'] * len(counties),
        addresses,
        absentee_voting_contact_phone_numbers
    )),
    columns = ['state', 'state abbreviation', 'locality', 'locality type', 'address', 'Phone']
)

In [5]:
standardized_df.head(50)

,state,state abbreviation,locality,locality type,address,Phone
0,Minnesota,MN,Aitkin County,County,"307 2nd St NW Rm 121 Aitkin, MN 56431",218-927-7354
1,Minnesota,MN,Anoka County,County,"2100 3rd Ave, Suite W130 Anoka, MN 55303-5031",763-324-1300
2,Minnesota,MN,Becker County,County,"915 Lake Ave Detroit Lakes, MN 56501",218-846-7311
3,Minnesota,MN,Beltrami County,County,"701 Minnesota Ave NW Ste 220 Bemidji, MN 5660...",218-333-8476
4,Minnesota,MN,Benton County,County,"531 Dewey St Foley, MN 56329-0129",320-968-5027
5,Minnesota,MN,Big Stone County,County,"20 2nd St SE Ste 103 Ortonville, MN 56278",320-839-6366
6,Minnesota,MN,Blue Earth County,County,"204 S 5th St Mankato, MN 56001",507-304-4341
7,Minnesota,MN,Brown County,County,"14 South State St New Ulm, MN 56073",507-233-6614
8,Minnesota,MN,Carlton County,County,"301 Walnut Ave Carlton, MN 55718",218-384-9127
9,Minnesota,MN,Carver County,County,"600 E Fourth St Chaska, MN 55318-2102",952-361-1941


In [6]:
standardized_df.tail(40)

,state,state abbreviation,locality,locality type,address,Phone
47,Minnesota,MN,Mille Lacs County,County,"635 2nd St SE Milaca, MN 56353",320-983-8310
48,Minnesota,MN,Morrison County,County,"213 SE 1st Ave Little Falls, MN 56345",320-632-0132
49,Minnesota,MN,Mower County,County,"201 1st St. NE, Ste 7 Austin, MN 55912",507-437-9535
50,Minnesota,MN,Murray County,County,"2500 28th St Slayton, MN 56172-0057",507-836-1154
51,Minnesota,MN,Nicollet County,County,"501 S Minnesota Ave St Peter, MN 56082",507-934-7806
52,Minnesota,MN,Nobles County,County,"315 10th St Worthington, MN 56187",507-295-5258
53,Minnesota,MN,Norman County,County,"16 E 3rd Ave Ada, MN 56510",218-784-5471
54,Minnesota,MN,Olmsted County,County,"151 4th St SE Rochester, MN 55904</div>",507-328-7650
55,Minnesota,MN,Otter Tail County,County,"510 Fir Ave W Fergus Falls, MN 56537",218-998-8040
56,Minnesota,MN,Pennington County,County,"101 Main Ave N Thief River Falls, MN 56701",218-683-7000


In [7]:
standardized_df.to_csv('mn_ballot_dropoff_locations.csv', index=False)